In [1]:
import pandas as pd
import numpy as np
import glob, os
import re

# Source

Stripped from site:
http://www.espn.com/college-football/team/schedule/_/id/2640/year/2017

- appended scores & schedule from 2008 - 2017
- `tsu_08_17.csv`

In [2]:
ls sports_data_raw/NCAA/

RU08_17.csv*  tsu08_11.csv*  tsu08_17.csv*  tsu12_17.csv*  uh08_17.csv*


## Load Data

In [3]:
path = 'sports_data_raw/NCAA/tsu08_17.csv'
df = pd.read_csv(path)

In [4]:
df.head()

,YEAR,DATE,OPPONENT,RESULT/TIME,RECORD/TICKETS
0,2017,"Sat, Aug 26",@Florida A&M,L29-7,0-1 (0-0)
1,2017,"Thu, Sept 7",vsHouston Baptist,L24-17,0-2 (0-0)
2,2017,"Sat, Sept 23",@Alabama A&M,L30-13,0-3 (0-1)
3,2017,"Fri, Sept 29",vsAlcorn State,L24-17,0-4 (0-2)
4,2017,"Sat, Oct 7",@Kennesaw St,L48-3,0-5 (0-2)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 5 columns):
YEAR              112 non-null int64
DATE              112 non-null object
OPPONENT          112 non-null object
RESULT/TIME       112 non-null object
RECORD/TICKETS    111 non-null object
dtypes: int64(1), object(4)
memory usage: 4.5+ KB


## Create slice with only needed columns

In [6]:
tsu1 = df[['DATE','YEAR','OPPONENT','RESULT/TIME']]

In [7]:
tsu1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
DATE           112 non-null object
YEAR           112 non-null int64
OPPONENT       112 non-null object
RESULT/TIME    112 non-null object
dtypes: int64(1), object(3)
memory usage: 3.6+ KB


## inspect data from Df

In [8]:
tsu1.YEAR.unique()  # need to convert to str

array([2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008])

In [9]:
tsu1.DATE.unique()  # need to strip, day, month numdate

array(['Sat, Aug 26', 'Thu, Sept 7', 'Sat, Sept 23', 'Fri, Sept 29',
       'Sat, Oct 7', 'Sat, Oct 14', 'Sat, Oct 28', 'Sat, Nov 4',
       'Sat, Nov 11', 'Sat, Nov 18', 'Sat, Nov 25', 'Sun, Sept 4',
       'Sat, Sept 10', 'Sat, Sept 17', 'Sat, Sept 24', 'Sat, Oct 1',
       'Sat, Oct 15', 'Sat, Oct 22', 'Sat, Oct 29     ', 'Sat, Nov 5',
       'Sat, Nov 12', 'Sat, Nov 19', 'Sat, Sept 5    ', 'Fri, Sept 11',
       'Sat, Sept 19', 'Sat, Sept 26 ', 'Thu, Oct 1', 'Sat, Oct 17',
       'Sat, Oct 24', 'Sat, Oct 31', 'Sat, Nov 7', 'Sat, Nov 14',
       'Sat, Nov 28', 'Sun, Aug 31', 'Thu, Sept 4', 'Sat, Sept 13',
       'Sat, Sept 20', 'Sat, Sept 27', 'Sat, Oct 4', 'Sat, Oct 18',
       'Sat, Oct 25', 'Sat, Nov 1', 'Sat, Nov 8', 'Sat, Nov 15',
       'Sat, Aug 31', 'Sat, Sept 14', 'Thu, Sept 19', 'Sat, Sept 28',
       'Sat, Oct 5', 'Sat, Oct 12', 'Sat, Oct 19', 'Sat, Oct 26',
       'Fri, Nov 1', 'Sat, Nov 9', 'Sat, Nov 16', 'Sat, Sept 1',
       'Sat, Sept 8', 'Sat, Sept 15', 'Sat, Sept 2

In [10]:
tsu1.OPPONENT.unique()  # need to strip @, vs, strip

array(['@Florida A&M  ', 'vsHouston Baptist ', '@Alabama A&M',
       'vsAlcorn State ', '@Kennesaw St', 'vsAlabama State',
       '@Grambling  ', '@Miss Valley St', 'vsSouthern',
       'vsAR-Pine Bluff*', 'vsPV A&M ', '@PV A&M', '@Houston Baptist ',
       'vsMiss Valley St', '@Alabama State', 'vsAlabama A&M',
       '@Alcorn State', 'vsJackson State', '@Sam Houston ', '@Southern ',
       'vsAR-Pine Bluff', 'vsGrambling', ' vsPV A&M ', 'vsBacone College',
       '@AR-Pine Bluff', '@Jackson State', '    vsAlabama State',
       '    @Miss Valley St', '    vsSouthern', '    vsFaith-Arkansas ',
       '    @Grambling', '    vsAlcorn State', '    @Alabama A&M',
       'vsPV A&M*', 'vsTexas College', 'vsCentral St*', '@Southern',
       'vsPV A&M', '@Sam Houston', 'vsAlcorn State', '@Grambling',
       'vsHoward', '@North Texas', 'vsSam Houston', 'vsTexas College ',
       'vsCentral St', '@UConn', 'vsTuskegee*', 'vsAlabama State*',
       '@UL Monroe', '@Texas State', '@Rutgers', 'vsAR-

In [11]:
tsu1['RESULT/TIME'].unique()  # need split  letter from score, strip extra stpaces

array([' L29-7', ' L24-17', 'L30-13 ', 'L48-3', 'L23-16', 'L50-24',
       'W38-21  ', 'L33-7  ', 'W24-10 ', '   L30-16  ', 'L29-25 ',
       'L24-20', 'W31-0', 'W31-27', 'W34-31', 'L23-20', 'L21-13',
       'L66-17', 'L26-10', 'W27-10', 'L47-28', 'L38-11', 'W63-0',
       'W24-20', 'L34-30', 'L41-23', 'W49-21', 'L40-21  ', 'Canceled',
       'L41-15', 'L65-13', 'L38-7', 'W37-35', 'W52-14', 'W30-16',
       'W45-23', 'L38-3', 'W20-16', 'L40-25', 'L38-37', 'L35-7', 'L30-20',
       'L15-10', 'L37-13', 'L55-17', 'L12-10', 'L34-2', 'W41-28',
       'L20-13', 'W23-17 OT', 'L31-24', 'L20-17', 'L40-6', 'W44-41',
       'L34-7', 'L45-35', 'L42-13', 'L50-6', 'L45-0', 'W23-20', 'L49-3',
       'L34-24', 'L34-3', ' L37-34', 'W49-6 ', 'W14-7', 'L58-13',
       'L43-29', 'L24-21', 'W42-11', 'L12-9', 'W29-15', 'L29-25', 'L42-6',
       'L16-14', 'L62-3', 'W32-9', 'L21-14', 'W21-7', 'W30-20', 'W21-18',
       'W38-7', 'W54-7', 'W41-34 OT', 'W20-13', 'W11-6', 'L17-7', 'L58-0',
       'W75-6', 'L52-18

# Clean data

## Date clean

In [12]:
tsu1['YEAR'] = tsu1['YEAR'].astype(str)  # change year value to str

/home/salas/miniconda3/envs/springboard/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
tsu1['full_date'] = tsu1['DATE'] + ' ' + tsu1['YEAR']

In [14]:
tsu1['full_date'].str.split(',').head()

0     [Sat,  Aug 26 2017]
1     [Thu,  Sept 7 2017]
2    [Sat,  Sept 23 2017]
3    [Fri,  Sept 29 2017]
4      [Sat,  Oct 7 2017]
Name: full_date, dtype: object

### create new colum for date, and cleanup for full_date

In [15]:
tsu1['Day']= tsu1['full_date'].str.split(',').apply(lambda x: x[0])
tsu1['full_date']= tsu1['full_date'].str.split(',').apply(lambda x: x[1])

In [16]:
tsu1.head()

,DATE,YEAR,OPPONENT,RESULT/TIME,full_date,Day
0,"Sat, Aug 26",2017,@Florida A&M,L29-7,Aug 26 2017,Sat
1,"Thu, Sept 7",2017,vsHouston Baptist,L24-17,Sept 7 2017,Thu
2,"Sat, Sept 23",2017,@Alabama A&M,L30-13,Sept 23 2017,Sat
3,"Fri, Sept 29",2017,vsAlcorn State,L24-17,Sept 29 2017,Fri
4,"Sat, Oct 7",2017,@Kennesaw St,L48-3,Oct 7 2017,Sat


### Convert DATE to datetime

In [17]:
tsu1['full_date'] = pd.to_datetime(tsu1['full_date'])

In [18]:
tsu1.head()

,DATE,YEAR,OPPONENT,RESULT/TIME,full_date,Day
0,"Sat, Aug 26",2017,@Florida A&M,L29-7,2017-08-26,Sat
1,"Thu, Sept 7",2017,vsHouston Baptist,L24-17,2017-09-07,Thu
2,"Sat, Sept 23",2017,@Alabama A&M,L30-13,2017-09-23,Sat
3,"Fri, Sept 29",2017,vsAlcorn State,L24-17,2017-09-29,Fri
4,"Sat, Oct 7",2017,@Kennesaw St,L48-3,2017-10-07,Sat


In [19]:
tsu = tsu1[['full_date','Day','OPPONENT','RESULT/TIME']]

In [20]:
tsu.head()

,full_date,Day,OPPONENT,RESULT/TIME
0,2017-08-26,Sat,@Florida A&M,L29-7
1,2017-09-07,Thu,vsHouston Baptist,L24-17
2,2017-09-23,Sat,@Alabama A&M,L30-13
3,2017-09-29,Fri,vsAlcorn State,L24-17
4,2017-10-07,Sat,@Kennesaw St,L48-3


## Reindex dataframe by date

- reindex the dataframe and sort the dates

In [21]:
tsu = tsu.set_index('full_date').sort_index(ascending=True)

In [22]:
tsu.head()

,Day,OPPONENT,RESULT/TIME
full_date,,,
2008-08-30,Sat,vsPV A&M*,L34-14
2008-09-06,Sat,@Arkansas State,L83-10
2008-09-13,Sat,@Shaw,W40-29
2008-09-20,Sat,@Texas College,W28-13
2008-09-27,Sat,vsConcordia Col.,W45-27


In [23]:
tsu.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 112 entries, 2008-08-30 to 2017-11-25
Data columns (total 3 columns):
Day            112 non-null object
OPPONENT       112 non-null object
RESULT/TIME    112 non-null object
dtypes: object(3)
memory usage: 3.5+ KB


# Create new column
- home_away given values of OPPONENT column

In [24]:
tsu['away_home'] = np.where(tsu['OPPONENT'].str.contains('@'),'away', 'home')

In [25]:
tsu.head()

,Day,OPPONENT,RESULT/TIME,away_home
full_date,,,,
2008-08-30,Sat,vsPV A&M*,L34-14,home
2008-09-06,Sat,@Arkansas State,L83-10,away
2008-09-13,Sat,@Shaw,W40-29,away
2008-09-20,Sat,@Texas College,W28-13,away
2008-09-27,Sat,vsConcordia Col.,W45-27,home


## remove extra str values from OPPONENT
- vs
- @


In [26]:
tsu['OPPONENT']= tsu['OPPONENT'].str.replace('vs','')

tsu['OPPONENT']= tsu['OPPONENT'].str.replace('@','')

In [27]:
tsu.head()

,Day,OPPONENT,RESULT/TIME,away_home
full_date,,,,
2008-08-30,Sat,PV A&M*,L34-14,home
2008-09-06,Sat,Arkansas State,L83-10,away
2008-09-13,Sat,Shaw,W40-29,away
2008-09-20,Sat,Texas College,W28-13,away
2008-09-27,Sat,Concordia Col.,W45-27,home


## create new col for RESULT/TIME

In [28]:
tsu['win_lost'] = np.where(tsu['RESULT/TIME'].str.contains('L'),'L', 'W')

In [29]:
tsu.head()

,Day,OPPONENT,RESULT/TIME,away_home,win_lost
full_date,,,,,
2008-08-30,Sat,PV A&M*,L34-14,home,L
2008-09-06,Sat,Arkansas State,L83-10,away,L
2008-09-13,Sat,Shaw,W40-29,away,W
2008-09-20,Sat,Texas College,W28-13,away,W
2008-09-27,Sat,Concordia Col.,W45-27,home,W


## remove extra str values from  RESULT/TIME
- L
- W

In [30]:
tsu['RESULT/TIME']= tsu['RESULT/TIME'].str.replace('L','')

tsu['RESULT/TIME']= tsu['RESULT/TIME'].str.replace('W','')
tsu['RESULT/TIME'] = tsu['RESULT/TIME'].str.strip()

In [31]:
tsu.head()

,Day,OPPONENT,RESULT/TIME,away_home,win_lost
full_date,,,,,
2008-08-30,Sat,PV A&M*,34-14,home,L
2008-09-06,Sat,Arkansas State,83-10,away,L
2008-09-13,Sat,Shaw,40-29,away,W
2008-09-20,Sat,Texas College,28-13,away,W
2008-09-27,Sat,Concordia Col.,45-27,home,W


## Rename columns

In [32]:
tsu.rename(columns={'OPPONENT':'Opponent',
                  'RESULT/TIME':'Result'}, inplace=True)

In [33]:
tsu.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 112 entries, 2008-08-30 to 2017-11-25
Data columns (total 5 columns):
Day          112 non-null object
Opponent     112 non-null object
Result       112 non-null object
away_home    112 non-null object
win_lost     112 non-null object
dtypes: object(5)
memory usage: 5.2+ KB


In [34]:
tsu['Day'].unique()

array(['Sat', 'Thu', 'Sun', 'Fri'], dtype=object)

In [35]:
tsu['Opponent'].unique()  # need to clean up

array(['PV A&M*', 'Arkansas State', 'Shaw', 'Texas College',
       'Concordia Col.', 'Texas State', 'Southern', 'Jackson State',
       'Alcorn State', 'Miss Valley St', 'Grambling*', 'AR-Pine Bluff',
       'PV A&M', 'UL Monroe', 'Rutgers', 'Grambling', 'AR-Pine Bluff* ',
       'UConn', 'Alabama A&M', 'Tuskegee*', 'Alabama State',
       'Alabama State*', 'Texas College ', 'Central St', 'North Texas',
       'Sam Houston', 'Howard', 'Central St*', ' PV A&M ',
       'Bacone College', '    Alabama State', '    Miss Valley St',
       '    Southern', '    Faith-Arkansas ', '    Grambling',
       '    Alcorn State', '    Alabama A&M', 'Houston Baptist ',
       'Sam Houston ', 'Southern ', 'Florida A&M  ', 'Alcorn State ',
       'Kennesaw St', 'Grambling  ', 'AR-Pine Bluff*', 'PV A&M '],
      dtype=object)

In [36]:
tsu['Result'].unique()

array(['34-14', '83-10', '40-29', '28-13', '45-27', '63-39', '45-14',
       '30-14', '30-29', '58-44', '33-7', '28-7', '17-7', '58-0', '75-6',
       '52-18', '42-0', '19-17', '51-21', '30-7', '47-33', '14-10',
       '30-25', '16-14', '62-3', '32-9', '21-14', '21-7', '30-20',
       '21-18', '38-7', '54-7', '41-34 OT', '20-13', '11-6', '37-34',
       '49-6', '14-7', '58-13', '43-29', '24-21', '42-11', '12-9',
       '29-15', '29-25', '42-6', '44-41', '34-7', '45-35', '42-13',
       '50-6', '45-0', '23-20', '49-3', '34-24', '34-3', '37-13', '55-17',
       '35-7', '12-10', '34-2', '41-28', '23-17 OT', '31-24', '20-17',
       '40-6', '37-35', '52-14', '30-16', '45-23', '38-3', '20-16',
       '40-25', '38-37', '15-10', '38-11', '63-0', '24-20', '34-30',
       '41-23', '49-21', '40-21', 'Canceled', '41-15', '65-13', '31-0',
       '31-27', '34-31', '21-13', '66-17', '26-10', '27-10', '47-28',
       '29-7', '24-17', '30-13', '48-3', '23-16', '50-24', '38-21',
       '24-10'], dtype=

In [37]:
tsu['away_home'].unique()

array(['home', 'away'], dtype=object)

In [38]:
tsu['win_lost'].unique()

array(['L', 'W'], dtype=object)

## cleanup Opponenet column
- remove '*' from values

In [39]:
tsu['Opponent'].unique()

array(['PV A&M*', 'Arkansas State', 'Shaw', 'Texas College',
       'Concordia Col.', 'Texas State', 'Southern', 'Jackson State',
       'Alcorn State', 'Miss Valley St', 'Grambling*', 'AR-Pine Bluff',
       'PV A&M', 'UL Monroe', 'Rutgers', 'Grambling', 'AR-Pine Bluff* ',
       'UConn', 'Alabama A&M', 'Tuskegee*', 'Alabama State',
       'Alabama State*', 'Texas College ', 'Central St', 'North Texas',
       'Sam Houston', 'Howard', 'Central St*', ' PV A&M ',
       'Bacone College', '    Alabama State', '    Miss Valley St',
       '    Southern', '    Faith-Arkansas ', '    Grambling',
       '    Alcorn State', '    Alabama A&M', 'Houston Baptist ',
       'Sam Houston ', 'Southern ', 'Florida A&M  ', 'Alcorn State ',
       'Kennesaw St', 'Grambling  ', 'AR-Pine Bluff*', 'PV A&M '],
      dtype=object)

In [40]:
tsu['Opponent']= tsu['Opponent'].str.replace('*','')
tsu['Opponent'] = tsu['Opponent'].str.strip()

In [41]:
tsu['Opponent'].value_counts(dropna=False)

Grambling          10
PV A&M             10
Alcorn State       10
Miss Valley St     10
AR-Pine Bluff      10
Southern           10
Jackson State       9
Alabama State       9
Alabama A&M         8
Texas College       4
Sam Houston         3
Central St          2
Houston Baptist     2
Texas State         2
Faith-Arkansas      1
Rutgers             1
Howard              1
UConn               1
Kennesaw St         1
UL Monroe           1
North Texas         1
Shaw                1
Bacone College      1
Concordia Col.      1
Arkansas State      1
Florida A&M         1
Tuskegee            1
Name: Opponent, dtype: int64

In [42]:
tsu.head()

,Day,Opponent,Result,away_home,win_lost
full_date,,,,,
2008-08-30,Sat,PV A&M,34-14,home,L
2008-09-06,Sat,Arkansas State,83-10,away,L
2008-09-13,Sat,Shaw,40-29,away,W
2008-09-20,Sat,Texas College,28-13,away,W
2008-09-27,Sat,Concordia Col.,45-27,home,W


## Save clean data to  to csv

In [43]:
tsu.to_csv('sports_data_clean/tsu_08_17_clean.csv')

In [44]:
ls sports_data_clean/

astros_08_17_clean.csv*       nfl_08_17_clean.csv*  rice_08_17_clean.csv*
dynamo_08_17_clean.csv*       nfl10clean.csv*       tsu_08_17_clean.csv*
hou_rockets_08_17_clean.csv*  nfl10yearsfull.csv*


In [45]:
## finish